I used a conda env with python 3.86

I installed spacy like this
conda install -c conda-forge spacy
conda install -c conda-forge spacy-lookups-data
python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import json
import spacy

In [3]:
!ls

EDA.ipynb                          adverse_media_training.csv.zip
LICENSE                            non_adverse_media_training.csv.zip
README.md


In [4]:
am = pd.read_csv('adverse_media_training.csv.zip')
nam = pd.read_csv('non_adverse_media_training.csv.zip')

### put the am and nam training data together, keep the labels - some articles from the nam file have an am label and maybe also vice versa

In [5]:
am.label.unique()

array([nan, 'am', 'delete', 'delete        ', 'random', 'nam', 'doubt',
       'neither', 'am, doubt', 'am '], dtype=object)

In [6]:
# keep only those with am label
am_confirmed = am.loc[(am.label == 'am') | (am.label == 'am ')]
am_confirmed.shape

(392, 11)

In [7]:
nam.label.unique()

array(['nam', nan, 'am', 'neither', 'random', 'doubt', 'delete'],
      dtype=object)

In [8]:
nam.loc[nam.label == 'am'].shape, am_confirmed.shape

((19, 11), (392, 11))

In [9]:
# Move rows with "am" from nam file to am df
am_confirmed = pd.concat([am_confirmed, nam.loc[nam.label == 'am']])

In [10]:
nam.label.unique()

array(['nam', nan, 'am', 'neither', 'random', 'doubt', 'delete'],
      dtype=object)

In [11]:
# keep only those with am label
nam_confirmed = nam.loc[(nam.label == 'nam') | (nam.label == 'random')]
nam_confirmed.shape

(288, 11)

In [12]:
am.loc[am.label == 'nam'].shape

(18, 11)

In [13]:
# Move rows with "nam" from am file to nam df
nam_confirmed = pd.concat([nam_confirmed, am.loc[am.label == 'nam']])
nam_confirmed.shape

(306, 11)

In [15]:
am_confirmed['is_adverse_media'] = 1
nam_confirmed['is_adverse_media'] = 0

In [19]:
# put am and nam into one DF
train = pd.concat([am_confirmed, nam_confirmed])

In [23]:
train.is_adverse_media.describe()

count    717.000000
mean       0.573222
std        0.494955
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: is_adverse_media, dtype: float64

### basic preprocessing: keep only needed columns, make sure no null values for important columns

In [30]:
# keep only needed columns
train = train.loc[:, ['entity_name', 'entity_type', 'url', 'article', 'full_response', 'explanation', 'title', 'is_adverse_media']]

train.describe(include='all')

,entity_name,entity_type,url,article,full_response,explanation,title,is_adverse_media
count,618,619,717,717,717,626,717,717.000000
unique,478,16,717,717,717,363,711,NaN
top,John McAfee,individual,https://www.thejakartapost.com/news/2013/09/21...,Nauru is banking on fields of mineral-rich smo...,[{'query': {'id': '1605055021641-edd6ff95a605a...,corruption,President Lopez Obrador urges review of U.S. d...,NaN
freq,8,176,1,1,1,34,2,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.573222
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.494955
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [34]:
# make sure all articles and labels (is_adverse_media) values are there
sum(train.article.isna()), sum(train.is_adverse_media.isna())

(0, 0)

In [33]:
# nlp = spacy.load("en_core_web_sm")